In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [4]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [5]:
EPOCHS = 50
BATCH_SIZE = 64

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                  train=True,
                  download=True,
                  transform = transforms.Compose([
                      transforms.RandomHorizontalFlip(),
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081))
                  ])),
    batch_size = BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                  train=False,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081))
                  ])),
    batch_size = BATCH_SIZE, shuffle=True)

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw
Processing...
Done!





/home/herbwood/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629434582/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [10]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [12]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss : 0.6263, Accuracy : 80.08%
[2] Test Loss : 0.4536, Accuracy : 85.43%
[3] Test Loss : 0.3423, Accuracy : 89.31%
[4] Test Loss : 0.2941, Accuracy : 91.38%
[5] Test Loss : 0.2598, Accuracy : 92.47%
[6] Test Loss : 0.2263, Accuracy : 93.13%
[7] Test Loss : 0.1952, Accuracy : 94.33%
[8] Test Loss : 0.1862, Accuracy : 94.32%
[9] Test Loss : 0.1608, Accuracy : 95.08%
[10] Test Loss : 0.1608, Accuracy : 95.16%
[11] Test Loss : 0.1489, Accuracy : 95.54%
[12] Test Loss : 0.1428, Accuracy : 95.47%
[13] Test Loss : 0.1362, Accuracy : 95.74%
[14] Test Loss : 0.1264, Accuracy : 96.17%
[15] Test Loss : 0.1164, Accuracy : 96.39%
[16] Test Loss : 0.1153, Accuracy : 96.43%
[17] Test Loss : 0.1136, Accuracy : 96.38%
[18] Test Loss : 0.1069, Accuracy : 96.84%
[19] Test Loss : 0.1083, Accuracy : 96.72%
[20] Test Loss : 0.1123, Accuracy : 96.49%
[21] Test Loss : 0.1044, Accuracy : 96.85%
[22] Test Loss : 0.1004, Accuracy : 96.96%
[23] Test Loss : 0.0961, Accuracy : 97.04%
[24] Test Loss : 0.0